In [1]:
import random
import time
import typing as tp
from dataclasses import dataclass

@dataclass
class MockResponse:
    headers: dict
    def raise_for_status(self): pass

class MockMapper:

    def to_native(self, uri: str) -> str:
        return uri

class NCEIBackendMock:
    def __init__(self):
        # Simulamos que tenemos un archivo de 50MB
        self.file_size = 50 * 1024 * 1024
        self.chunk_count = 50 # Lo dividiremos en 50 partes de 1MB

    def size(self, *, uri: str) -> int:
        return self.file_size

    def read_chunk(self, *, uri: str, chunk_size: int = 1024 * 1024) -> tp.Iterable[bytes]:
        print(f"\n[Mock] Iniciando stream para: {uri}")
        for _ in range(self.chunk_count):
            latencia = random.uniform(0.01, 0.15)
            time.sleep(latencia)  # Simulamos latencia de red
            yield b"0" * chunk_size # Enviamos un MB de "datos" dummy

# Mock simple del Datasource para que funcione con smart_download
class DatasourceMock:
    def __init__(self):
        self.backend = NCEIBackendMock()
        self.mapper = MockMapper()
        self.mountpoint = "/ncei_data"

    def get_size(self, uri: str) -> int:
        return self.backend.size(uri=uri)

    def stream(self, uri: str) -> tp.Iterable[bytes]:
        return self.backend.read_chunk(uri=uri)

In [2]:
import pathlib as pl
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

def smart_download(ds, remote_uris: list[str], local_dest: str | pl.Path, max_workers: int = 4):
    """
    Descarga archivos usando streaming.
    """
    local_root = pl.Path(local_dest)

    # Ejecución
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Barra de progreso global (por archivos)
        with tqdm(total=len(remote_uris), desc="Progreso Global", unit="file", position=0, leave=True) as main_pbar:
            # Asignamos una posición a cada hilo para que las barras de tqdm se apilen
            futures = [
                executor.submit(_download_one, ds, local_root, uri, i % max_workers + 1)
                for i, uri in enumerate(remote_uris)
            ]
            for future in as_completed(futures):
                main_pbar.update(1)

def _download_one(ds, local_root, uri: str, position: int):
    # 1. Intentar obtener el tamaño para la barra de progreso
    total_size = ds.get_size(uri=uri)

    # 2. Preparar ruta local
    # (Uso de lstrip para asegurar que sea relativa al root local)
    relative_path = uri.split(ds.mountpoint)[-1].lstrip("/")
    target_path = local_root / relative_path
    target_path.parent.mkdir(parents=True, exist_ok=True)

    # 3. Descarga con streaming
    with tqdm(
        total=total_size,
        unit='B',
        unit_scale=True,
        desc=uri.split('/')[-1],
        leave=False, # La barra desaparece al terminar para no saturar
        position=position # Evita que las barras se solapen en multihilo
    ) as pbar:
        with open(target_path, "wb") as f:
            for chunk in ds.stream(uri=uri):
                f.write(chunk)
                pbar.update(len(chunk))

In [3]:
from tqdm.notebook import tqdm
import pathlib as pl

# 1. Instanciar el mock
ds_mock = DatasourceMock()

# 2. Definir archivos ficticios para descargar
archivos_ficticios = [
    "/ncei_data/sensor_v1_2024.nc",
    "/ncei_data/sensor_v2_2024.nc",
    "/ncei_data/sensor_v3_2024.nc",
    "/ncei_data/sensor_v4_2024.nc",
    "/ncei_data/sensor_v5_2024.nc",
    "/ncei_data/metadata_global.nc",
]

# 3. Ejecutar la descarga inteligente
print("Iniciando prueba de descarga concurrente con streaming...")
smart_download(
    ds=ds_mock,
    remote_uris=archivos_ficticios,
    local_dest="./test_mock_download",
    max_workers=3
)

Iniciando prueba de descarga concurrente con streaming...


Progreso Global:   0%|          | 0/6 [00:00<?, ?file/s]

sensor_v1_2024.nc:   0%|          | 0.00/52.4M [00:00<?, ?B/s]


[Mock] Iniciando stream para: /ncei_data/sensor_v1_2024.nc


sensor_v3_2024.nc:   0%|          | 0.00/52.4M [00:00<?, ?B/s]

...


[Mock] Iniciando stream para: /ncei_data/sensor_v4_2024.nc


sensor_v5_2024.nc:   0%|          | 0.00/52.4M [00:00<?, ?B/s]


[Mock] Iniciando stream para: /ncei_data/sensor_v5_2024.nc


metadata_global.nc:   0%|          | 0.00/52.4M [00:00<?, ?B/s]


[Mock] Iniciando stream para: /ncei_data/metadata_global.nc


In [3]:
import tfg.storage

In [4]:
gc = tfg.storage.use_gcs_cloud(bucket="gcp-public-data-goes-16", cache_file="hola.json", expire_after=100.0)
ls = tfg.storage.use_local_drive(root_path=".")

content = gc.list(prefix="/ABI-L2-CMIPF/2020/319/20")
# data = gc.load(uri=content[0])

# ls.save(uri=content[0], data=data)
# print(type(data))

In [ ]:
ds = tfg.storage.use_aws_cloud(bucket="noaa-goes16", cache_file="hola.json", expire_after=100.0)

content = ds.list(prefix="/ABI-L2-CMIPF/2020/319/20")
data = ds.load(uri=content[1])

ls.save(uri=content[1], data=data)
print(type(data))

In [ ]:
ls = tfg.storage.use_local_drive(root_path=".")
content = ls.list(prefix="/ABI-L2-CMIPF/2020/319/20")
data = ls.load(uri=content[0])
print(type(data))

In [ ]:
gd = tfg.storage.use_google_drive()
gd.save(uri=content[0], data=data)
print(type(data))

In [ ]:
nc = tfg.storage.use_ncei_archive(base_url="https://www.ncei.noaa.gov/data/gridsat-goes/access/goes", cache_file="hola.json", expire_after=100.0)
ls = tfg.storage.use_local_drive(root_path=".")

content = nc.list(prefix="/2007/08/")
data = nc.load(uri=content[0])

ls.save(uri=content[0], data=data)
print(type(data))

In [4]:
import goesdl.gridsat

filter = goesdl.gridsat.GridSatProductLocatorGC(scene="F", origins="G12", versions="v01")

print(filter)
print(filter.get_base_url("HTTP"))
# filter.get_datetime("HTTP")
# filter.get_paths("HTTP")
filter.match("HTTP")
help(filter)

GridSatProductLocatorGC(name='GOES', origins=['goes12'], versions=['v01'], file_date_format='%Y.%m.%d.%H%M', file_date_pattern='\\d{4}\\.\\d{2}\\.\\d{2}\\.\\d{4}', file_prefix='GridSat', path_date_format='%Y/%m', path_prefix='goes/')
('https://www.ncei.noaa.gov/data/gridsat-goes/access/',)
Help on GridSatProductLocatorGC in module goesdl.gridsat.locator_gc object:

class GridSatProductLocatorGC(goesdl.gridsat.locator.GridSatProductLocator)
 |  GridSatProductLocatorGC(
 |      scene: str,
 |      origins: str | list[str],
 |      versions: str | list[str] = 'v01'
 |  ) -> None
 |
 |  Represent the GridSat-GOES/CONUS imagery dataset product locator.
 |
 |  This class implements the `GridSatProductLocator` abstract class
 |  for the GridSat-GOES/CONUS (Geostationary Operational Environmental
 |  Satellites - GOES/CONUS) dataset product locator.
 |
 |  Instances of this class are responsible for generating a list of
 |  folder paths based on the dataset's directory structure and naming
 | 

In [33]:
filter = goesdl.gridsat.GridSatProductLocatorB1(versions="v02r01")


# filter.get_base_url("")

In [10]:
import contextlib
import locale
import os
import typing as tp

LangType = tp.Literal["en", "es"]

SUPPORTED_LANG = set(tp.get_args(LangType))


def _get_lang(lang_param: str | None) -> str:
    """
    Determina el idioma para mostrar la ayuda según la precedencia
    establecida.

    Args:
        lang_param: Parámetro 'lang' pasado a la función factoría

    Returns:
        'en' para inglés o 'es' para español

    Raises:
        ValueError: Si lang_param no es None y no es 'en' o 'es'
    """
    # 1. Verificar el parámetro lang
    if lang_param is not None:
        if lang_param not in SUPPORTED_LANG:
            raise ValueError(
                f"El idioma '{lang_param}' no es válido. "
                "Use 'en' para inglés o 'es' para español."
            )
        print(f"Idioma seleccionado por parámetro: {lang_param}")
        return lang_param

    # 2. Verificar la variable de entorno GOESDL_LANG
    if env_lang := os.environ.get("GOESDL_LANG"):
        env_lang = env_lang.strip().lower()
        if env_lang in SUPPORTED_LANG:
            print(f"Idioma seleccionado por variable de entorno: {env_lang}")
            return env_lang

        # Si la variable existe pero tiene un valor no válido,
        # ¿continuamos con el siguiente método o notificamos de la configuración incorrecta?

    # 3. Intentar detectar el idioma del sistema operativo

    with contextlib.suppress(Exception):
        sys_lang, _ = locale.getdefaultlocale()
        if sys_lang:
            lang_code = sys_lang.split("_")[0].lower()
            if lang_code in SUPPORTED_LANG:
                print(f"Idioma detectado del sistema operativo: {lang_code}")
                return lang_code

    # 4. Por defecto: inglés
    return "en"


# Función de prueba para verificar el comportamiento
def test_lang_selection() -> None:
    """Función para probar la selección de idioma"""
    print("=== Pruebas de selección de idioma ===")

    # Guardar estado original de variables de entorno
    original_env = os.environ.get("GOESDL_LANG")

    # Test 1: Parámetro explícito
    print("\n1. Con parámetro 'es':")
    print(f"   Resultado: {_get_lang('es')}")

    # Test 2: Parámetro 'en'
    print("\n2. Con parámetro 'en':")
    print(f"   Resultado: {_get_lang('en')}")

    # Test 3: Variable de entorno
    print("\n3. Con variable GOESDL_LANG='es':")
    os.environ["GOESDL_LANG"] = "es"
    print(f"   Resultado: {_get_lang(None)}")

    # Test 4: Parámetro inválido
    print("\n4. Con parámetro inválido 'fr':")
    try:
        _get_lang("fr")
    except ValueError as e:
        print(f"   Excepción: {e}")

    # Restaurar variable de entorno
    if original_env:
        os.environ["GOESDL_LANG"] = original_env
    else:
        del os.environ["GOESDL_LANG"]

    print("\n=== Fin de pruebas ===")


if __name__ == "__main__":
    # Ejecutar pruebas
    test_lang_selection()

    # Mostrar idioma detectado actualmente
    print(f"Idioma detectado actualmente: {_get_lang(None)}")

=== Pruebas de selección de idioma ===

1. Con parámetro 'es':
Idioma seleccionado por parámetro: es
   Resultado: es

2. Con parámetro 'en':
Idioma seleccionado por parámetro: en
   Resultado: en

3. Con variable GOESDL_LANG='es':
Idioma seleccionado por variable de entorno: es
   Resultado: es

4. Con parámetro inválido 'fr':
   Excepción: El idioma 'fr' no es válido. Use 'en' para inglés o 'es' para español.

=== Fin de pruebas ===
Idioma detectado del sistema operativo: en
Idioma detectado actualmente: en


C:\Users\Spacewatch\AppData\Local\Temp\ipykernel_13264\928364179.py:48: DeprecationWarning: 'locale.getdefaultlocale' is deprecated and slated for removal in Python 3.15. Use setlocale(), getencoding() and getlocale() instead.
  sys_lang, _ = locale.getdefaultlocale()


In [42]:
import locale

locale.getlocale() 

('English_United Kingdom', 'utf8')

In [46]:
 locale.normalize(locale.getlocale()[0]) 

'English_United Kingdom'

In [47]:
locale.getdefaultlocale()

C:\Users\Spacewatch\AppData\Local\Temp\ipykernel_8744\793819832.py:1: DeprecationWarning: 'locale.getdefaultlocale' is deprecated and slated for removal in Python 3.15. Use setlocale(), getencoding() and getlocale() instead.
  locale.getdefaultlocale()


('en_GB', 'cp65001')